In [ ]:
import numpy as np
import pandas as pd
from numpy import where

In [ ]:
applications = pd.read_csv(
    r'C:\Users\PC\Desktop\tech_stuff\Studying\Python\GoIT_course\HW 4-5\applications(2.0).csv',
    sep=None,
    header=0,
    engine='python',
    names=['applied_date', 'amount', 'age', 'gender', 'industry', 'status', 'ext_rating', 'ed_level', 'loc', 'id']
)

applications_df = pd.DataFrame(applications)

print(applications_df.head())

In [ ]:
industries = pd.read_csv(
    r"C:\Users\PC\Desktop\tech_stuff\Studying\Python\GoIT_course\HW 4-5\industries(2.0).csv",
    sep=None,
    header=0,
    engine='python',
    names=['industry', 'ind_score']
)

industries_df = pd.DataFrame(industries)

print(industries_df.head())

In [ ]:
#1 Прибери дублікати applicant_id
# В полі 'External Rating' заповни відсутні значення нулями
# В полі 'Education level' заповни відсутні значення текстом “Середня”

app_no_dubs_df = applications_df.drop_duplicates(
    subset='id',
    keep='first',
    inplace=False
)

print(app_no_dubs_df.head())

In [ ]:
apps_ext_rat_ed_level_no_nan_df = app_no_dubs_df.copy()

apps_ext_rat_ed_level_no_nan_df[['ext_rating', 'ed_level']] = app_no_dubs_df[['ext_rating', 'ed_level']].fillna(
    value={'ext_rating': 0, 'ed_level': 'Середня'},
    inplace=False
)

print(apps_ext_rat_ed_level_no_nan_df)

In [ ]:
# Answer:

nan_count = apps_ext_rat_ed_level_no_nan_df[['ext_rating', 'ed_level']].isna().sum()
cleaned_applications_df = apps_ext_rat_ed_level_no_nan_df.copy()
print(cleaned_applications_df)

In [ ]:
#2 Додай до цього DataFrame дані з файлу industries.csv, а саме, рейтинги індустрій.

merged_df = pd.merge(
    cleaned_applications_df, industries,
    how='left',
    on='industry',
)

In [ ]:
# alternative - to use inner 

merged_df12 = pd.merge(
    cleaned_applications_df, industries,
    how='inner',
    on='industry',
)

print(merged_df12.head(30))

In [ ]:
# Answer:

print(merged_df.head())

In [ ]:
#3 Розрахуй рейтинг заявки

merged_df_rating = merged_df.copy()

merged_df_rating['app_date_proper'] = pd.to_datetime(merged_df_rating['applied_date'], errors='coerce')
print(merged_df_rating.head())
print(merged_df_rating.dtypes)

In [ ]:
merged_df_rating['cond1_b'] = merged_df_rating['amount'].isna()
merged_df_rating['cond2_b'] = merged_df_rating['ext_rating'] == 0
merged_df_rating['cond3_b'] = (merged_df_rating['age'] >= 35) & (merged_df_rating['age'] <= 55)
merged_df_rating['cond4_b'] = merged_df_rating['loc'] == 'Київ чи область'
merged_df_rating['cond5_b'] = merged_df_rating['ext_rating'] >= 7
merged_df_rating['cond6_b'] = merged_df_rating['app_date_proper'].dt.weekday < 5
merged_df_rating['cond7_b'] = merged_df_rating['ext_rating'] >= 7
merged_df_rating['cond8_b'] = merged_df_rating['ext_rating'] <= 2
print(merged_df_rating.head())

In [ ]:
merged_df_rating['cond1'] = merged_df_rating['cond1_b'] * 1
merged_df_rating['cond2'] = merged_df_rating['cond2_b'] * 1
merged_df_rating['cond3'] = merged_df_rating['cond3_b'] * 20
merged_df_rating['cond4'] = merged_df_rating['cond4_b'] * 10
merged_df_rating['cond5'] = merged_df_rating['cond5_b'] * 20
merged_df_rating['cond6'] = merged_df_rating['cond6_b'] * 20
merged_df_rating['cond7'] = merged_df_rating['cond7_b'] * 20
merged_df_rating['cond8'] = merged_df_rating['cond8_b'] * -20

print(merged_df_rating[['cond1', 'cond2', 'cond3', 'cond4', 'cond5', 'cond6', 'cond7', 'cond8', 'ind_score']].head(30))

In [ ]:
merged_df_rating['app_rating'] = where(
    (merged_df_rating['cond1'] == 1) | (merged_df_rating['cond2'] == 1),
    0,
    merged_df_rating['cond3'] + 
    merged_df_rating['cond4'] + 
    merged_df_rating['cond5'] +
    merged_df_rating['cond6'] + 
    merged_df_rating['cond7'] + 
    merged_df_rating['cond8'] + 
    merged_df_rating['ind_score']
)

In [ ]:
# Answer:

print(merged_df_rating.head(40))

In [ ]:
#4 В результуючій таблиці залиш лише заявки з рейтингом більше нуля, ці заявки вважатимуться прийнятими.

merged_df_rating['app_rating'] = where(
    merged_df_rating['app_rating'] > 100,
    100,
    merged_df_rating['app_rating']
)

print(merged_df_rating['app_rating'].head(50))

In [ ]:
merged_df_rating = merged_df_rating.sort_values('app_rating', ascending = False)
print(merged_df_rating['app_rating'].head(50))

In [ ]:
merged_df_4 = merged_df_rating[merged_df_rating['app_rating'] > 0]
print(merged_df_4['app_rating'].tail(20))

In [ ]:
merged_df_4 = merged_df_4.copy()


merged_df_4.drop(columns=[
    'cond3', 'cond4', 'cond5', 'cond6', 'cond7', 'cond8',
    'cond1', 'cond2', 'cond1_b', 'cond2_b', 'applied_date',
    'cond3_b', 'cond4_b', 'cond5_b', 'cond6_b', 'cond7_b', 'cond8_b'
], inplace=True)

In [ ]:
# Answer:

print(merged_df_4.head(40))

In [ ]:
#5 Дані з результуючої таблиці згрупуй за тижнем подачі заявки, 
# та виведи середній рейтинг прийнятих заявок у кожен тиждень.

merged_df_5 = merged_df_4.copy()
merged_df_5['app_rating'] = merged_df_4['app_rating'].mean()

print(merged_df_5.head(5))

In [ ]:
merged_df_4['app_week'] = merged_df_4['app_date_proper'].dt.strftime('%Y-%W')
merged_df_4 = merged_df_4.sort_values('app_week')

print(merged_df_4[['app_date_proper', 'app_week']].head())

In [ ]:
merged_df_5 = merged_df_4.groupby('app_week')['app_rating'].mean().reset_index()
merged_df_5['app_rating'] = merged_df_5['app_rating'].round(2)

In [ ]:
# Answer:

print(merged_df_5)

In [ ]:
merged_df_4.to_csv(
    'hw_45',
    sep=';',
    index=False)